In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [6]:
# start of project, see if single excitation wavefunction can be used to fine tune the parameters of the hartree fock method, 

In [5]:
import numpy as np
import pyscf
from pyscf import gto, scf, ao2mo

# ============================================================================
# Setup:  H2 molecule and basis
# ============================================================================
mol = gto.Mole()
mol.atom = '''
H 0.0 0.0 0.0
H 0.0 0.0 0.74
'''
mol.basis = 'aug-cc-pvdz'
mol.unit = 'angstrom'
mol.symmetry = False
mol.build()

print("=" * 70)
print("H2 molecule with aug-cc-pVDZ basis")
print("=" * 70)
print(f"Number of AOs: {mol.nao}")
print(f"Number of electrons: {mol.nelectron}")
print()

# ============================================================================
# Step 1: Run RHF to get reference determinant |Ψ0⟩
# ============================================================================
mf = scf.RHF(mol)
mf.conv_tol = 1e-10
mf.kernel()

E_HF = mf.e_tot
print(f"RHF converged:  {mf.converged}")
print(f"RHF total energy E0 = {E_HF:.10f} Hartree")
print()

# ============================================================================
# Extract HF orbital energies and MO coefficients
# ============================================================================
mo_energy = mf.mo_energy  # Orbital energies ε_p
mo_coeff = mf.mo_coeff    # MO coefficient matrix (AO → MO)
nocc = mol.nelectron // 2  # Number of doubly occupied orbitals
nvir = mol.nao - nocc      # Number of virtual orbitals

print(f"Number of occupied orbitals: {nocc}")
print(f"Number of virtual orbitals: {nvir}")
print()

print("Orbital energies (Hartree):")
print("Occupied:")
for i in range(nocc):
    print(f"  MO {i}: ε = {mo_energy[i]:.6f}")
print("Virtual:")
for a in range(nocc, mol.nao):
    print(f"  MO {a}:  ε = {mo_energy[a]:.6f}")
print()

# ============================================================================
# Step 2: Transform integrals to MO basis
# ============================================================================
# One-electron integrals (kinetic + nuclear attraction)
h1e_ao = mol.intor('int1e_kin') + mol.intor('int1e_nuc')
h1e_mo = mo_coeff. T @ h1e_ao @ mo_coeff

# Two-electron integrals in MO basis (pq|rs) in physicist notation
# PySCF ao2mo returns integrals in chemist notation (pq|rs) = [pr|qs]_chem
# We use general transformation
eri_mo = ao2mo.kernel(mol, mo_coeff)
# Restore to 4-index array (physicist notation:  (pq|rs))
eri_mo = ao2mo.restore(1, eri_mo, mol.nao)  # Shape: (nmo, nmo, nmo, nmo)

print("Integrals transformed to MO basis.")
print()

# ============================================================================
# Step 3: Build Fock matrix in MO basis
# ============================================================================
# F_pq = h_pq + Σ_j [ (pq|jj) - (pj|jq) ]
# sum over occupied j
fock_mo = h1e_mo. copy()
for p in range(mol.nao):
    for q in range(mol.nao):
        for j in range(nocc):
            fock_mo[p, q] += 2.0 * eri_mo[p, q, j, j] - eri_mo[p, j, j, q]

print("Fock matrix diagonal (should match orbital energies):")
for p in range(mol.nao):
    print(f"  F_{p}{p} = {fock_mo[p, p]:.6f},  ε_{p} = {mo_energy[p]:.6f},  diff = {abs(fock_mo[p, p] - mo_energy[p]):.2e}")
print()

# ============================================================================
# Step 4: Choose one single excitation i → a
# ============================================================================
# For H2 with 2 electrons:  nocc = 1 (HOMO = 0), nvir starts at index 1 (LUMO)
# Choose HOMO → LUMO excitation
i = nocc - 1  # HOMO (occupied)
a = nocc      # LUMO (virtual)

print("=" * 70)
print(f"Chosen single excitation: i = {i} (HOMO) → a = {a} (LUMO)")
print("=" * 70)
print()

# ============================================================================
# Step 5: Compute matrix elements for 2×2 Hamiltonian
# ============================================================================
# H_00 = E_HF (reference energy)
H_00 = E_HF

# H_01 = ⟨Ψ0|H|Ψ_i^a⟩ = F_{ia}
V = fock_mo[i, a]

# H_11 = E0 + (ε_a - ε_i)
Delta = mo_energy[a] - mo_energy[i]
H_11 = E_HF + Delta

print("Matrix elements:")
print(f"  H_00 = E_HF = {H_00:.10f} Hartree")
print(f"  H_11 = E_HF + (ε_a - ε_i) = {H_11:.10f} Hartree")
print(f"  Δ = ε_a - ε_i = {Delta:.10f} Hartree")
print(f"  V = F_ia = {V:.10e} Hartree")
print()

# Check Brillouin theorem (V should be ~0 for canonical HF)
print("Brillouin theorem check:")
if abs(V) < 1e-8:
    print(f"  |V| = {abs(V):.2e} ≈ 0 ✓ (singles decouple from HF reference)")
else:
    print(f"  |V| = {abs(V):.2e} ≠ 0 (singles mix with reference)")
print()

# ============================================================================
# Step 6: Build and diagonalize 2×2 Hamiltonian matrix
# ============================================================================
H_2x2 = np. array([
    [H_00, V],
    [V,    H_11]
])

print("2×2 Hamiltonian matrix:")
print(H_2x2)
print()

# Diagonalize
eigenvalues, eigenvectors = np. linalg.eigh(H_2x2)

print("=" * 70)
print("Diagonalization results")
print("=" * 70)
print(f"Eigenvalue 1 (lower): {eigenvalues[0]:.10f} Hartree")
print(f"Eigenvalue 2 (upper): {eigenvalues[1]:.10f} Hartree")
print()

print("Eigenvectors (columns = eigenstates):")
print("  State 1 (lower):")
print(f"    c0 (HF component)      = {eigenvectors[0, 0]:.6f}")
print(f"    c1 (single excitation) = {eigenvectors[1, 0]:.6f}")
print(f"    |c0|^2 = {eigenvectors[0, 0]**2:.6f}")
print(f"    |c1|^2 = {eigenvectors[1, 0]**2:.6f}")
print()
print("  State 2 (upper):")
print(f"    c0 (HF component)      = {eigenvectors[0, 1]:.6f}")
print(f"    c1 (single excitation) = {eigenvectors[1, 1]:.6f}")
print(f"    |c0|^2 = {eigenvectors[0, 1]**2:.6f}")
print(f"    |c1|^2 = {eigenvectors[1, 1]**2:.6f}")
print()

# ============================================================================
# Step 7: Interpretation
# ============================================================================
print("=" * 70)
print("Interpretation")
print("=" * 70)
print(f"HF energy:          {E_HF:.10f} Hartree")
print(f"CI lower energy:   {eigenvalues[0]:.10f} Hartree")
print(f"Energy correction: {eigenvalues[0] - E_HF:.10e} Hartree")
print()
if abs(V) < 1e-8:
    print("Since V ≈ 0 (Brillouin theorem holds), the 2×2 Hamiltonian is diagonal.")
    print("The lower eigenvalue equals E_HF, and singles do not improve the")
    print("ground-state energy. This demonstrates why CIS does not recover")
    print("correlation energy for the ground state.")
else:
    print("V ≠ 0: singles mix with the HF reference, lowering the ground-state")
    print("energy within this 2-state subspace.")
print()

# ============================================================================
# Optional: Compare with full CI
# ============================================================================
print("=" * 70)
print("Optional: Full CI comparison (FCI in full basis)")
print("=" * 70)
from pyscf import fci

cisolver = fci.FCI(mf)
E_FCI, civec = cisolver.kernel()
print(f"FCI energy:  {E_FCI:.10f} Hartree")
print(f"Correlation energy (FCI - HF): {E_FCI - E_HF:.10f} Hartree")
print()
print("The 2-state CI model recovers only a tiny fraction of correlation because")
print("ground-state correlation for closed-shell systems requires double excitations.")
print("=" * 70)

H2 molecule with aug-cc-pVDZ basis
Number of AOs: 18
Number of electrons: 2

converged SCF energy = -1.12877827534965
RHF converged:  True
RHF total energy E0 = -1.1287782753 Hartree

Number of occupied orbitals: 1
Number of virtual orbitals: 17

Orbital energies (Hartree):
Occupied:
  MO 0: ε = -0.592790
Virtual:
  MO 1:  ε = 0.061494
  MO 2:  ε = 0.066958
  MO 3:  ε = 0.231549
  MO 4:  ε = 0.285138
  MO 5:  ε = 0.285138
  MO 6:  ε = 0.416597
  MO 7:  ε = 0.458703
  MO 8:  ε = 0.458703
  MO 9:  ε = 0.558573
  MO 10:  ε = 0.717539
  MO 11:  ε = 1.242664
  MO 12:  ε = 1.604557
  MO 13:  ε = 1.604557
  MO 14:  ε = 2.083357
  MO 15:  ε = 2.243895
  MO 16:  ε = 2.243895
  MO 17:  ε = 3.708548

Integrals transformed to MO basis.

Fock matrix diagonal (should match orbital energies):
  F_00 = -0.592790,  ε_0 = -0.592790,  diff = 8.57e-09
  F_11 = 0.061494,  ε_1 = 0.061494,  diff = 1.50e-09
  F_22 = 0.066958,  ε_2 = 0.066958,  diff = 2.72e-10
  F_33 = 0.231549,  ε_3 = 0.231549,  diff = 3.86e-